In [1]:
import pandas as pd

train_df = pd.read_csv("Train_Data.csv")
test_df = pd.read_csv("Test_Data.csv")

print('Train Data Shape: ', train_df.shape)
print('Test Data Shape: ', test_df.shape)

train_df.head()

Train Data Shape:  (1966, 10)
Test Data Shape:  (312, 9)


,SEQN,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN,age_group
0,73564.0,61.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91,Adult
1,73568.0,26.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85,Adult
2,73576.0,16.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14,Adult
3,73577.0,32.0,1.0,2.0,28.9,104.0,NaN,84.0,16.15,Adult
4,73580.0,38.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92,Adult


In [2]:
#changing the labels to numeric

train_df['age_group'] = train_df['age_group'].replace({'Adult': 0, 'Senior': 1})

C:\Users\ASUS\AppData\Local\Temp\ipykernel_21192\1315309828.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['age_group'] = train_df['age_group'].replace({'Adult': 0, 'Senior': 1})


In [3]:
print('Missing values in training data:', train_df.isnull().sum())
print('Missing values in testing data:', test_df.isnull().sum())

Missing values in training data: SEQN         12
RIDAGEYR      9
RIAGENDR     18
PAQ605       13
BMXBMI       18
LBXGLU       13
DIQ010       18
LBXGLT       11
LBXIN         9
age_group    14
dtype: int64
Missing values in testing data: SEQN        2
RIDAGEYR    3
RIAGENDR    2
PAQ605      1
BMXBMI      1
LBXGLU      1
DIQ010      1
LBXGLT      2
LBXIN       1
dtype: int64


In [4]:
from sklearn.impute import SimpleImputer

X_train = train_df.drop(columns=['SEQN', 'age_group'])
y_train = train_df['age_group']

X_test = test_df.drop(columns=['SEQN'])

imputer = SimpleImputer(strategy='mean')

In [5]:
X_train_imp = pd.DataFrame(imputer.fit_transform(X_train),  columns=X_train.columns)
X_test_imp = pd.DataFrame(imputer.transform(X_test),  columns=X_test.columns)

Just checking

In [6]:
print("Non-numeric columns in X_train:")
print(X_train.select_dtypes(exclude=['number']).columns)

print("\nNon-numeric columns in X_test:")
print(X_test.select_dtypes(exclude=['number']).columns)

Non-numeric columns in X_train:
Index([], dtype='object')

Non-numeric columns in X_test:
Index([], dtype='object')


In [7]:
print('Train data shape after imputatiomn: ', X_train_imp.shape)
print('Test data shape after imputatiomn: ', X_test_imp.shape)

print('Missing labels in train data: ', y_train.isnull().sum())


Train data shape after imputatiomn:  (1966, 8)
Test data shape after imputatiomn:  (312, 8)
Missing labels in train data:  14


In [8]:
for col in ['RIAGENDR', 'PAQ605', 'DIQ010']:
    X_train_imp[col] = X_train_imp[col].astype('category')
    X_test_imp[col] = X_test_imp[col].astype('category')

X_train_ohe = pd.get_dummies(X_train_imp)
X_test_ohe = pd.get_dummies(X_test_imp)

X_train_ohe, X_test_ohe = X_train_ohe.align(X_test_ohe, join='left', axis=1, fill_value=0)

In [9]:
valid_data = y_train.notna()

X_train_ohe = X_train_ohe[valid_data]
y_train = y_train[valid_data]

In [10]:
from sklearn.model_selection import train_test_split

X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_ohe, y_train, test_size=0.2, random_state=42, stratify=y_train)

Model

In [11]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train_final, y_train_final)

RandomForestClassifier(random_state=42)

In [12]:
y_val_pred = model.predict(X_val)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix

print('Classification Report: ', classification_report(y_val, y_val_pred))

Classification Report:                precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       328
         1.0       1.00      1.00      1.00        63

    accuracy                           1.00       391
   macro avg       1.00      1.00      1.00       391
weighted avg       1.00      1.00      1.00       391



In [14]:
print('Confusion Matrix: ', confusion_matrix(y_val, y_val_pred))

Confusion Matrix:  [[328   0]
 [  0  63]]


Prediction of test cases

In [15]:
test_pred = model.predict(X_test_ohe)

submission = pd.DataFrame({'age_group': test_pred.astype(int)})

In [16]:
submission.to_csv('submission.csv', index=False)